In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install import-ipynb
!pip3 install cuda-python

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import math
import pickle
import time
import sys

import import_ipynb
import AttackClassificationModel

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
importing Jupyter notebook from AttackClassificationModel.ipynb
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

# Data Pre-Processing

In [3]:
df_traffic = AttackClassificationModel.get_files("../../botiot")

# Get DDoS
df_ddos, label_ddos = AttackClassificationModel.get_attack(0, df_traffic)

# Get DoS
df_dos, label_dos = AttackClassificationModel.get_attack(1, df_traffic)

# Get Reconaissance
df_rec, label_rec = AttackClassificationModel.get_attack(2, df_traffic)

# Get Theft
df_theft, label_theft = AttackClassificationModel.get_attack(3, df_traffic)

In [4]:
# Normalize with the min-max scaler
scaler = MinMaxScaler()
df_ddos_norm = scaler.fit_transform(df_ddos)
df_dos_norm = scaler.fit_transform(df_dos)
df_rec_norm = scaler.fit_transform(df_rec)
df_theft_norm = scaler.fit_transform(df_theft)

# Sets

In [5]:
# Train
len_ddos_train = int(0.7 * len(df_ddos_norm))
X_ddos_train = df_ddos_norm[:len_ddos_train]

len_dos_train = int(0.7 * len(df_dos_norm))
X_dos_train = df_dos_norm[:len_dos_train]

X_train = np.concatenate([X_ddos_train, X_dos_train])
Y_train = np.concatenate([label_ddos[:len_ddos_train], label_dos[:len_dos_train]])

In [6]:
# Test
len_ddos_test = len_ddos_train + int(0.15 * len(df_ddos_norm))
X_ddos_test = df_ddos_norm[len_ddos_train : len_ddos_test]

len_dos_test = len_ddos_train + int(0.15 * len(df_dos_norm))
X_dos_test = df_dos_norm[len_dos_train : len_dos_test]

X_rec_test = df_rec_norm

label_theft = []
X_theft_test = df_theft_norm
for i in range(len(X_theft_test)):
    label_theft.append(2)

X_test_opt = np.concatenate([X_ddos_test, X_dos_test])
Y_test_opt = np.concatenate([label_ddos[len_ddos_train : len_ddos_test], label_dos[len_dos_train : len_dos_test]])

X_test = np.concatenate([X_ddos_test, X_dos_test, X_rec_test, X_theft_test])
Y_test = np.concatenate([label_ddos[len_ddos_train : len_ddos_test], label_dos[len_dos_train : len_dos_test], label_rec, label_theft])

In [7]:
# Validation
X_ddos_val = df_ddos_norm[len_ddos_test:]
X_dos_val = df_dos_norm[len_dos_test:]

X_val = np.concatenate([X_ddos_val, X_dos_val])
Y_val = np.concatenate([label_ddos[len_ddos_test:], label_dos[len_dos_test:]])

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test_opt = np.reshape(X_test_opt, (X_test_opt.shape[0], X_test_opt.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

samples, feature, depth = X_train.shape

# Model

In [9]:
# Initial values
batch_size = 50
number_features = 35
learning_rate = 0.008
epochs = 1

# Dictionary
dict_params = { 'learning_rate': learning_rate, 'batch_size': round(batch_size), 'epochs': round(epochs) }
pbounds = { 'learning_rate': (0.000001, 0.001), 'batch_size': (10, 30), 'epochs': (1, 50) }

In [10]:
cnn_model = AttackClassificationModel.CNN_botiot(feature, depth)
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

# Hyperparameter Optimization

In [11]:
def maximize_training(X_train = X_train,
                      Y_train = Y_train, 
                      X_val = X_val, 
                      Y_val = Y_val, 
                      X_test = X_test_opt, 
                      Y_test = Y_test_opt, 
                      learning_rate = learning_rate, 
                      epochs = epochs, 
                      batch_size = batch_size,
                      reduce_lr = reduce_lr):
    
    nadam = optimizers.Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
    model = AttackClassificationModel.CNN_botiot(feature, depth)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
    
    history = model.fit(X_train, Y_train, 
                        epochs = dict_params['epochs'], 
                        batch_size = dict_params['batch_size'], 
                        validation_data = (X_val, Y_val),
                        callbacks = [reduce_lr],
                        verbose = 0)

    scores = model.evaluate(X_test, Y_test)
    return scores[1]

In [12]:
'''
# Apply Bayesian optimization to choose the best hyperparameters

opt = BayesianOptimization(f = maximize_training,
                           pbounds = pbounds,
                           verbose = 2, 
                           random_state = 1)

opt_start = time.time()

opt.maximize(init_points = 5, n_iter = 5)

opt_end = time.time()
opt_time = opt_end - opt_start
print("Optimization time:", opt_time)

learning_rate = opt.max['params']['learning_rate']
epochs = round(opt.max['params']['epochs'])
batch_size = round(opt.max['params']['batch_size'])
'''

'\n# Apply Bayesian optimization to choose the best hyperparameters\n\nopt = BayesianOptimization(f = maximize_training,\n                           pbounds = pbounds,\n                           verbose = 2, \n                           random_state = 1)\n\nopt_start = time.time()\n\nopt.maximize(init_points = 5, n_iter = 5)\n\nopt_end = time.time()\nopt_time = opt_end - opt_start\nprint("Optimization time:", opt_time)\n\nlearning_rate = opt.max[\'params\'][\'learning_rate\']\nepochs = round(opt.max[\'params\'][\'epochs\'])\nbatch_size = round(opt.max[\'params\'][\'batch_size\'])\n'

# Classify Attacks

In [13]:
# Training step with the best hyperparameters
learning_rate = 0.0006855342808963628
epochs = 1
batch_size = 21
opt_time = 3915.1859345436096

In [14]:
nadam = optimizers.Nadam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
cnn_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])

for i in range(1):
    print("Iteration " + str(i))
    
    # Train
    cnn_model, train_time = AttackClassificationModel.train(X_train, Y_train, X_val, Y_val, epochs, batch_size, reduce_lr, cnn_model)
    
    # Test
    Y_pred, test_time = AttackClassificationModel.test(X_test, cnn_model)
    print(Y_pred)
    print(Y_test)
    
    # Multi classification metrics
    labels = ['DDoS', 'DoS', 'Unknown']
    acc, f1, pre, rec = AttackClassificationModel.get_scores(Y_test, Y_pred, labels)
    
    # Print results
    AttackClassificationModel.print_results(learning_rate,
                                            epochs,
                                            batch_size,
                                            X_train,
                                            X_val,
                                            X_test,
                                            opt_time,
                                            train_time,
                                            test_time,
                                            acc,
                                            f1,
                                            pre,
                                            rec,
                                            Y_test,
                                            Y_pred,
                                            "CNN",
                                            "Results/botiot.txt")
    
    print("")

Iteration 0
Train on 2503818 samples, validate on 343080 samples
2503818/2503818 [==============================] - ETA: 0s - loss: 6.0657e-04 - accuracy: 0.9999

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2503818/2503818 [==============================] - 505s 202us/sample - loss: 6.0657e-04 - accuracy: 0.9999 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - lr: 0.0010
Training time: 505.62210178375244


/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Testing time: 30.75585126876831
[9.9999994e-01 1.2292906e-35]
[True, False]
[9.9999994e-01 1.2290844e-35]
[True, False]
[9.9999994e-01 1.2288687e-35]
[True, False]
[9.9999994e-01 1.2192497e-35]
[True, False]
[9.9999994e-01 1.5039354e-29]
[True, False]
[9.9999994e-01 1.5036600e-29]
[True, False]
[9.9999994e-01 1.5034075e-29]
[True, False]
[9.9999994e-01 1.5031552e-29]
[True, False]
[9.9999994e-01 1.5028916e-29]
[True, False]
[9.9999994e-01 1.5026278e-29]
[True, False]
[9.9999994e-01 1.5023757e-29]
[True, False]
[9.9999994e-01 1.5021464e-29]
[True, False]
[9.9999994e-01 1.5018715e-29]
[True, False]
[9.9999994e-01 1.5016079e-29]
[True, False]
[9.9999994e-01 1.5013673e-29]
[True, False]
[9.9999994e-01 1.5011040e-29]
[True, False]
[9.9999994e-01 1.5008405e-29]
[True, False]
[9.9999994e-01 1.5005772e-29]
[True, False]
[9.9999994e-01 1.5003253e-29]
[True, False]
[9.9999994e-01 1.4999934e-29]
[True, False]
[9.9999994e-01 1.4997417e-29]
[True, False]
[9.9999994e-01 1.4996159e-29]
[True, False]


ValueError: Found input variables with inconsistent numbers of samples: [821147, 100]

# Naive Bayes Model

In [ ]:
'''
# Model
nb_model = GaussianNB()

for i in range(1):
    print("Iteration " + str(i))
    
    # Train
    nb_model, train_time = AttackClassificationModel.train_nb_knn(X_train, Y_train, X_val, nb_model)
    
    # Test
    Y_pred, test_time = AttackClassificationModel.test(X_test, nb_model)
    
    # Multi classification metrics
    labels = ['DDoS', 'DoS', 'Unknown']
    acc, f1, pre, rec = AttackClassificationModel.get_scores(Y_test, Y_pred, labels)
    
    # Print results
    AttackClassificationModel.print_results(learning_rate,
                                            epochs,
                                            batch_size,
                                            X_train,
                                            X_val,
                                            X_test,
                                            opt_time,
                                            train_time,
                                            test_time,
                                            acc,
                                            f1,
                                            pre,
                                            rec,
                                            Y_test,
                                            Y_pred,
                                            "NB",
                                            "Results/botiot.txt)
    
    print("")
'''

# KNN Model

In [ ]:
'''
# Model
knn_model = KNeighborsClassifier(n_neighbors = 50)

for i in range(1):
    print("Iteration " + str(i))
    
    # Train
    knn_model, train_time = AttackClassificationModel.train_nb_knn(X_train, Y_train, X_val, knn_model)
    
    # Test
    Y_pred, test_time = AttackClassificationModel.test(X_test, knn_model)
    
    # Multi classification metrics
    labels = ['DDoS', 'DoS', 'Unknown']
    acc, f1, pre, rec = AttackClassificationModel.get_scores(Y_test, Y_pred, labels)
    
    # Print results
    AttackClassificationModel.print_results(learning_rate,
                                            epochs,
                                            batch_size,
                                            X_train,
                                            X_val,
                                            X_test,
                                            opt_time,
                                            train_time,
                                            test_time,
                                            acc,
                                            f1,
                                            pre,
                                            rec,
                                            Y_test,
                                            Y_pred,
                                            "NB",
                                            "Results/botiot.txt)
    
    print("")
'''